# OutOfRangeNullTransformer
This notebook shows the functionality in the `OutOfRangeNullTransformer` class. This transformer works in a similar way to the `CappingTransformer` but it does not apply capping, instead any values that are outside the 'cap range' will be set to `null` instead of the limits of that range. <br>
For more examples of setting the limits to set null values, refer to the `CappingTransformer` examples notebook, as `OutOfRangeNullTransformer` works the same way.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

In [2]:
import tubular
from tubular.capping import OutOfRangeNullTransformer

In [3]:
tubular.__version__

'0.3.0'

## Load California housing dataset from sklearn

In [4]:
cali = fetch_california_housing()
cali_df = pd.DataFrame(cali["data"], columns=cali["feature_names"])
cali_df["AveOccup"] = cali_df["AveOccup"].sample(frac=0.99, random_state=123)

In [5]:
cali_df.head()

MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25

In [6]:
cali_df.dtypes

MedInc        float64
HouseAge      float64
AveRooms      float64
AveBedrms     float64
Population    float64
AveOccup      float64
Latitude      float64
Longitude     float64
dtype: object

## Simple usage
First we will demonstrate the case where the user has pre determined the values to cap at, and codes these directly into the transformer.

### Initialising OutOfRangeNullTransformer

The `OutOfRangeNullTransformer` should be initialised (in the same way as the `CappingTransformer`) by specifying either a `capping_values` dict or a `quantiles` dict, but not both. <br>
These must be a `dict` where each key is a column to apply capping to and the items are lists of length 2, containing either `None` or numeric values;
- in the case of `capping_values` the user directly specifies the values to cap at giving a lower and upper value here 
- or if `quantiles` is specified the values in each list should be the quantiles to cap at (these will be determined when running the `fit` method) 
- if a `None` value is present then the relevant min or max capping is not applied. <br>

In the example below both upper and lower capping ranges will be considered, any values above `0.66` or below `0.44` will be replaced with `np.NaN` values. If `0.66` was replaced with `None` then only values below `0.44` would be set to null.

In [7]:
cap_1 = OutOfRangeNullTransformer(capping_values={"AveOccup": [1, 5.5]})

### OutOfRangeNullTransformer fit
In the case where the user specifies `capping_values` then there is no need to use the `fit` method. This is only required is the user specifies the `quantiles` argument when initialising the transformer. In fact if `capping_values` were specified and `fit` is called a warning will be generated.

### OutOfRangeNullTransformer transform
Like the `CappingTransformer` if the transformer is applied to non-numeric columns then an exception will be raised. <br>
Below we demonstrate any values outside the set range of `[0.44, 0.66]` being set to null.

In [8]:
cali_df["AveOccup"].quantile([0, 0.2, 0.8, 1.0])

0.0      0.692308
0.2      2.339853
0.8      3.424850
1.0    599.714286
Name: AveOccup, dtype: float64

In [9]:
((cali_df["AveOccup"] < 1) | (cali_df["AveOccup"] > 5.5)).sum()

182

In [10]:
cali_df["AveOccup"].isnull().sum()

206

In [11]:
cali_df_2 = cap_1.transform(cali_df)

In [12]:
cali_df_2["AveOccup"].quantile([0, 1.0])

0.0    1.060606
1.0    5.500000
Name: AveOccup, dtype: float64

In [13]:
cali_df_2["AveOccup"].isnull().sum()

388

In [14]:
((cali_df_2["AveOccup"] < 1) | (cali_df_2["AveOccup"] > 5.5)).sum()

0